In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

In [16]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [23]:
# Drop columns only if they exist
columns_to_drop = ['RowNumber', 'CustomerId', 'Surname']
existing_columns = [col for col in columns_to_drop if col in data.columns]
if existing_columns:
    data = data.drop(existing_columns, axis=1)
    print(f"Dropped columns: {existing_columns}")
else:
    print("Columns already dropped or don't exist")
data.head()

Columns already dropped or don't exist


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [1]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

NameError: name 'LabelEncoder' is not defined

In [40]:
oneohot_encoder_geo=OneHotEncoder()
geo_encoder = oneohot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder.toarray()
oneohot_encoder_geo.get_feature_names_out(['Geography'])

geo_encoded_df = pd.DataFrame(geo_encoder.toarray() , columns=oneohot_encoder_geo.get_feature_names_out(['Geography']))

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)




KeyError: "None of [Index(['Geography'], dtype='object')] are in the [columns]"

In [41]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [42]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(oneohot_encoder_geo,file)

In [ ]:
X = data.drop('Exited',axis=1)
y = data['Exited']
X_train,X_test,y_train,y_test = train_test_split(X,y ,test_size=0.2,random_state=42)

scaler = StandardScaler()
X_train-scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)



,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [ ]:

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(oneohot_encoder_geo,file)



with open('scaler.pkl','wb') as file:
    pickle.dump(scaler    ,file)

ANN Implementaion

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [62]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),#Hl1
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
    
])

In [68]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.losses.BinaryCrossentropy()
loss

model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [80]:
log_dir='logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

tensorflow_callback=TensorBoard(log_dir , histogram_freq=1)

In [81]:
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5080 - accuracy: 0.7945 - val_loss: 0.4881 - val_accuracy: 0.8035
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5082 - accuracy: 0.7945 - val_loss: 0.4885 - val_accuracy: 0.8035
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5076 - accuracy: 0.7945 - val_loss: 0.4885 - val_accuracy: 0.8035
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5079 - accuracy: 0.7945 - val_loss: 0.4887 - val_accuracy: 0.8035
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5082 - accuracy: 0.7945 - val_loss: 0.4892 - val_accuracy: 0.8035
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5081 - accuracy: 0.7945 - val_loss: 0.4883 - val_accuracy: 0.8035
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5082 - accuracy: 0.7945 - val_loss: 0.4881 - val_accuracy: 0.8035

In [76]:
model.save('model.h5')

c:\Users\Hasan\Desktop\Desktop Projects\Data sceince Projects\machin learning\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [77]:
%load_ext tensorboard


In [82]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 13716), started 0:00:34 ago. (Use '!kill 13716' to kill it.)